In [2]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv("data/Table O FY.csv")

df["Quantity"].fillna(0, inplace=True)

# 使用 pivot_table 函数进行数据透视
pivot_df = df.pivot_table(
    index=["Jurisdiction", "iso_3166_2", "YearDur", "Year"],
    columns="Fuel",
    values="Quantity",
    aggfunc="first",
).reset_index()

# 删除year = #VALUE!的行
pivot_df = pivot_df[pivot_df["Year"] != "#VALUE!"]

# 将结果保存为新的 CSV 文件
csv_path = "data/Table_O_FY_pivot.csv"
pivot_df.to_csv(csv_path, index=False)

In [1]:
import pandas as pd

# 读取电力数据文件
electricity_df = pd.read_csv("data/Table_O_FY_pivot.csv")

# 读取二氧化碳排放数据文件
emissions_df = pd.read_csv("data/Emissions by state and territory year elec.csv")

# 删除AUS的数据
electricity_df = electricity_df[electricity_df["Jurisdiction"] != "AUS"]

# 将二氧化碳数据从宽格式转换为长格式
emissions_melted = emissions_df.melt(
    id_vars=["Financial Year"],
    value_vars=["ACT", "NSW", "NT", "QLD", "SA", "TAS", "VIC", "WA"],
    var_name="Jurisdiction",
    value_name="CO2_Emissions",
)

# 重命名Financial Year列为Year以便合并
emissions_melted = emissions_melted.rename(columns={"Financial Year": "Year"})

# 处理"Not available"值，将其转换为NaN
emissions_melted["CO2_Emissions"] = pd.to_numeric(
    emissions_melted["CO2_Emissions"], errors="coerce"
)

# 合并电力数据和二氧化碳数据
merged_df = pd.merge(
    electricity_df, emissions_melted, on=["Jurisdiction", "Year"], how="left"
)

# 显示合并后的数据前几行
print("合并后的数据前几行:")
print(merged_df.head())

# 显示数据的基本信息
print("\n数据基本信息:")
print(merged_df.info())

# 显示各州的二氧化碳数据整合情况
print("\n各州二氧化碳数据整合情况:")
co2_summary = merged_df.groupby("Jurisdiction")["CO2_Emissions"].agg(
    ["count", "min", "max", "mean"]
)
print(co2_summary)

# 保存整合后的数据到新文件
merged_df.to_csv("data/merged_electricity_emissions_data.csv", index=False)
print("\n数据已保存到 'merged_electricity_emissions_data.csv'")

# 可选：显示每个州的年份范围和数据完整性
print("\n各州数据年份范围和完整性:")
for state in merged_df["Jurisdiction"].unique():
    state_data = merged_df[merged_df["Jurisdiction"] == state]
    years = state_data["Year"].unique()
    co2_missing = state_data["CO2_Emissions"].isna().sum()
    print(
        f"{state}: 年份范围 {years.min()}-{years.max()}, 数据行数: {len(state_data)}, 缺失CO2数据: {co2_missing}"
    )

合并后的数据前几行:
  Jurisdiction iso_3166_2  YearDur  Year  Bagasse, wood   Biogas  Black coal  \
0          NSW     AU-NSW  2012-13  2012        425.400  413.400   56798.600   
1          NSW     AU-NSW  2013-14  2013        461.700  420.600   55819.495   
2          NSW     AU-NSW  2014-15  2014        551.100  472.100   52562.200   
3          NSW     AU-NSW  2015-16  2015        601.734  444.606   55334.499   
4          NSW     AU-NSW  2016-17  2016        696.468  406.210   55967.139   

   Brown coal  Geothermal     Hydro  Large-scale solar PV  Natural gas  \
0         0.0         0.0  3298.089                 0.000     5168.700   
1         0.0         0.0  2721.109                10.635     5528.300   
2         0.0         0.0  1770.988                60.176     4528.390   
3         0.0         0.0  3170.201               399.511     4628.561   
4         0.0         0.0  3290.647               567.558     3330.808   

   Oil products  Other  Small-scale solar PV      Total  Total 